## Librerias

import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata


In [11]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [12]:
# Fijar directorio
DATA_DIR = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\raw")
DATA_OUT = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed")

In [13]:
# Lista todos los archivos .csv
csv_files = sorted(DATA_DIR.glob("*.csv"))
print(f"Se listan {len(csv_files)} archivos a procesar:")
for f in csv_files:
    print("-", f.name)

## Lectura de datos
data_raw = []
cols_by_file = {}
for file in csv_files:
    df = pd.read_csv(file, sep="|")
    df["file_origin"] = file.name
    data_raw.append(df)
    cols_by_file[file.name] = set(df.columns)
    
print("\nproceso cargado!")

print("\nUniendo datos!")
raw_all = pd.concat(data_raw, ignore_index=True)
print("Filas totales:", len(raw_all))
print("Columnas:", list(raw_all.columns))

Se listan 34 archivos a procesar:
- data_alquiler_departamentos_adondevivir_Lima Centro.csv
- data_alquiler_departamentos_adondevivir_Lima Este.csv
- data_alquiler_departamentos_adondevivir_Lima Moderna.csv
- data_alquiler_departamentos_adondevivir_Lima Norte.csv
- data_alquiler_departamentos_adondevivir_Lima Sur.csv
- data_alquiler_departamentos_adondevivir_Lima Top.csv
- data_alquiler_departamentos_urbania_Lima Centro.csv
- data_alquiler_departamentos_urbania_Lima Este.csv
- data_alquiler_departamentos_urbania_Lima Norte.csv
- data_alquiler_departamentos_urbania_Lima Sur.csv
- data_alquiler_departamentos_urbania_Lima Top.csv
- data_venta_casas_adondevivir_Lima Centro.csv
- data_venta_casas_adondevivir_Lima Este.csv
- data_venta_casas_adondevivir_Lima Moderna.csv
- data_venta_casas_adondevivir_Lima Norte.csv
- data_venta_casas_adondevivir_Lima Sur.csv
- data_venta_casas_adondevivir_Lima Top.csv
- data_venta_casas_urbania_Lima Centro.csv
- data_venta_casas_urbania_Lima Este.csv
- data_

In [14]:
print("\nLimpieza de datos!")

## Limpieza de datos - duplicados Otros. 

dups = raw_all["enlace"].duplicated(keep=False)
print("Registros duplicados por enlace:", dups.sum())

# ¿Cuántos enlaces únicos?
print("Enlaces únicos:", raw_all["enlace"].nunique())

## removeremos duplicados.
raw_all_nd = raw_all.drop_duplicates(subset=["enlace"], keep="first") 
print("se tiene solo enlaces unicos:", raw_all_nd["enlace"].nunique())


Limpieza de datos!
Registros duplicados por enlace: 210
Enlaces únicos: 51058
se tiene solo enlaces unicos: 51058


In [15]:
raw_all_nd.columns

Index(['fuente', 'inmueble', 'operacion', 'zona', 'distrito', 'direccion',
       'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin'],
      dtype='object')

In [29]:
valores = {
    "Brena": "Breña",
    "Jesus Maria": "Jesús María",
    "San Martin De Porres": "San Martín de Porres",
    "Lurin": "Lurín",
    "Ancon": "Ancón",
    "Santa Maria Del Mar": "Santa María del Mar",
    "Villa Maria Del Triunfo": "Villa María del Triunfo",
}

s = (raw_all_nd["distrito"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True))

raw_all_nd["distrito2"] = s.replace(valores)   # crea la nueva columna

raw_all_nd["distrito_norm"] = raw_all_nd["distrito2"].str.lower().str.strip()
raw_all_nd["direccion_norm"] = raw_all_nd["direccion"].str.lower().str.strip()

raw_all_nd["distrito_en_direccion"] = raw_all_nd.apply(
    lambda row: row["distrito_norm"] in row["direccion_norm"],
    axis=1
)

C:\Users\PC\AppData\Local\Temp\ipykernel_29840\4271788306.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito2"] = s.replace(valores)   # crea la nueva columna
C:\Users\PC\AppData\Local\Temp\ipykernel_29840\4271788306.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito_norm"] = raw_all_nd["distrito2"].str.lower().str.strip()
C:\Users\PC\AppData\Local\Temp\ipykernel_29840\4271788306.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [47]:
cols = ["distrito_2", "direccion", 
        "distrito_norm", "direccion_norm", "distrito_en_direccion"]
raw_all_nd[raw_all_nd["distrito_en_direccion"]==False][cols]

,distrito_2,direccion,distrito_norm,direccion_norm,distrito_en_direccion
69,Breña,"JR HUASCAR CUADRA 21\nJesús María, Lima",breña,"jr huascar cuadra 21\njesús maría, lima",False
70,Breña,"HERMILIO VALDIZAN\nResidencial San Felipe, Jes...",breña,"hermilio valdizan\nresidencial san felipe, jes...",False
71,Breña,"av arnaldo marquez 1440\nJesús María, Lima",breña,"av arnaldo marquez 1440\njesús maría, lima",False
72,Breña,"Avenida Mariategui al 1400\nJesús María, Lima",breña,"avenida mariategui al 1400\njesús maría, lima",False
73,Breña,"Jirón Diego de Almagro\nJesús María, Lima",breña,"jirón diego de almagro\njesús maría, lima",False
...,...,...,...,...,...
40267,Villa Maria del Triunfo,Jr. José Olaya Mz. D6 Lt. 12\nSanta Rosa De Be...,villa maria del triunfo,jr. josé olaya mz. d6 lt. 12\nsanta rosa de be...,False
40268,Villa Maria del Triunfo,Villa María Del Triunfo\nVilla María del Triun...,villa maria del triunfo,villa maría del triunfo\nvilla maría del triun...,False
40269,Villa Maria del Triunfo,"Castilla 245\nVilla María del Triunfo, Lima",villa maria del triunfo,"castilla 245\nvilla maría del triunfo, lima",False
40270,Villa Maria del Triunfo,"Castilla 245\nVilla María del Triunfo, Lima",villa maria del triunfo,"castilla 245\nvilla maría del triunfo, lima",False


In [78]:
canon = {
    # canónico : [alias que podrían aparecer en la dirección]
    "Ate Vitarte": ["ate vitarte", "ate"],
    "Breña": ["breña", "brena"],
    "Carabayllo": ["carabayllo"],
    "Chaclacayo": ["chaclacayo"],
    "El Agustino": ["el agustino", "agustino"],
    "Independencia": ["independencia"],
    "Jesús María": ["jesus maria", "jesús maria", "jesus maría"], ## Jesús María
    "Chosica Lurigancho": [ "chosica"],
    "Lurín": ["lurin", "lurín"],
    "Pachacamac": ["pachacamac", "pachacámac"],
    "Pucusana": ["pucusana"],
    "Puente Piedra": ["puente piedra"],
    "Punta Negra": ["punta negra"],
    "Rimac": ["rimac", "rímac"],
    "San Juan de Miraflores": ["san juan de miraflores", "sjm"],
    "San Luis": ["san luis"],
    "San Martin de Porres": ["san martin de porres", "san martín de porres", "smp"],
    "Villa Maria del Triunfo": ["villa maria del triunfo", "villa maría del triunfo", "vmt"],
    "Ancón": ["ancon", "ancón"],
    "Lince": ["lince"],
    "Santa Rosa": ["santa rosa"],
    "Barranco": ["barranco"],
    "Magdalena": ["magdalena del mar", "magdalena"],
    "Punta Hermosa": ["punta hermosa"],
    "Santiago de Surco": ["santiago de surco", "surco"],
    "San Juan de Lurigancho": ["san juan de lurigancho"],
    "Lima Cercado" : ["cercado", "cercado de lima", "lima cercado"],
    "Los Olivos" : ["los olivos"],
    "Comas" : ["comas"],
    "San Bartolo" : ["san bartolo"], 
    'Villa El Salvador' : ["villa el salvador"],
    "Santa Maria del Mar": ["santa maria del mar"],
    "San Isidro" : ["san isidro"],
    "La Molina" : ["la molina"]
}

In [79]:

def strip_accents(s):
    if not isinstance(s, str):
        return s
    return ''.join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
raw_all_nd["dist_norm"] = raw_all_nd["distrito_2"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)

# precompila patrones regex alias → canónico
patterns = []
for canon_name, aliases in canon.items():
    for alias in aliases:
        pat = re.compile(rf"\b{re.escape(strip_accents(alias))}\b")
        patterns.append((canon_name, pat))

def candidates_from_address(addr_norm: str):
    if not isinstance(addr_norm, str):
        return []
    hits = []
    for canon_name, pat in patterns:
        if pat.search(addr_norm):
            hits.append(canon_name)
    # quitar duplicados preservando orden
    return  list(dict.fromkeys(hits))

raw_all_nd["dist_candidates"] = raw_all_nd["dir_norm"].map(candidates_from_address)
raw_all_nd["distrito_candidato"] = raw_all_nd["dist_candidates"].str.get(0) 

C:\Users\PC\AppData\Local\Temp\ipykernel_29840\1625289358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
C:\Users\PC\AppData\Local\Temp\ipykernel_29840\1625289358.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["dist_norm"] = raw_all_nd["distrito_2"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
C:\Users\PC\AppData\Local\Temp\

In [80]:
raw_all_nd[raw_all_nd["distrito_en_direccion"]==False][cols + ["distrito_candidato"]].head()

,distrito_2,direccion,distrito_norm,direccion_norm,distrito_en_direccion,distrito_candidato
69,Breña,"JR HUASCAR CUADRA 21\nJesús María, Lima",breña,"jr huascar cuadra 21\njesús maría, lima",False,Jesús María
70,Breña,"HERMILIO VALDIZAN\nResidencial San Felipe, Jes...",breña,"hermilio valdizan\nresidencial san felipe, jes...",False,Jesús María
71,Breña,"av arnaldo marquez 1440\nJesús María, Lima",breña,"av arnaldo marquez 1440\njesús maría, lima",False,Jesús María
72,Breña,"Avenida Mariategui al 1400\nJesús María, Lima",breña,"avenida mariategui al 1400\njesús maría, lima",False,Jesús María
73,Breña,"Jirón Diego de Almagro\nJesús María, Lima",breña,"jirón diego de almagro\njesús maría, lima",False,Jesús María


In [81]:
raw_all_nd["distrito_3"] = np.where(
    raw_all_nd["distrito_en_direccion"]==False, raw_all_nd["distrito_candidato"], raw_all_nd["distrito_2"]
)

C:\Users\PC\AppData\Local\Temp\ipykernel_29840\2097689565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito_3"] = np.where(


In [85]:
raw_all_nd["distrito_3"].value_counts()

distrito_3
Miraflores                 9513
Santiago de Surco          9140
San Isidro                 5715
La Molina                  3566
San Borja                  2676
San Miguel                 2393
Barranco                   2068
Surquillo                  1576
Chorrillos                 1475
Jesús María                1230
Magdalena                  1149
Lima Cercado               1108
Pueblo Libre               1039
Lince                       881
Ate Vitarte                 727
Los Olivos                  589
San Martin de Porres        572
La Victoria                 570
Punta Hermosa               510
San Juan de Lurigancho      503
Comas                       435
San Bartolo                 411
San Juan de Miraflores      368
Carabayllo                  295
Breña                       264
Lurín                       232
Rimac                       189
Chosica Lurigancho          187
Villa El Salvador           181
Chaclacayo                  161
Santa Maria del Mar         1

In [92]:

data_p1 = raw_all_nd

cols_obj =['fuente', 'inmueble', 'operacion', 'distrito', 'distrito_3', 'zona', 'direccion', 'precio',
            'soles', 'dolares', 'variacion', 'mantenimiento', 'caracteristica',
            'area', 'dormitorio', 'baños', 'estacionamientos', 'detalle', 'enlace',
            'fecha', 'file_origin', 'distrito_candidato', 'distrito_norm',
            'direccion_norm', 'distrito_en_direccion']

data_p2 = data_p1[data_p1["distrito_candidato"].notna()][cols_obj].copy()

data_vivienda = pd.concat([data_p1], axis=0)

def normalizar(texto: str) -> str:
    # 1. Pasar a minúsculas
    texto = texto.lower()
    # 2. Quitar acentos
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    # 3. Quitar dobles espacios y limpiar
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


data_vivienda["distrito_oficial"] = data_vivienda["distrito_3"] ## .apply(normalizar)

In [93]:
data_vivienda

,fuente,inmueble,operacion,zona,distrito,direccion,precio,soles,dolares,variacion,...,distrito_norm,direccion_norm,distrito_en_direccion,distrito_oficial,distrito2,dir_norm,dist_norm,dist_candidates,distrito_candidato,distrito_3
0,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,lima cercado,"avenida venezuela 2575\nlima cercado, lima",True,Lima Cercado,Lima Cercado,"avenida venezuela 2575 lima cercado, lima",lima cercado,[Lima Cercado],Lima Cercado,Lima Cercado
1,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,NaN,...,lima cercado,"av. venezuela 2575\nlima cercado, lima",True,Lima Cercado,Lima Cercado,"av. venezuela 2575 lima cercado, lima",lima cercado,[Lima Cercado],Lima Cercado,Lima Cercado
2,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,NaN,...,lima cercado,"avenida colonial 3046\nla colonial, lima cercado",True,Lima Cercado,Lima Cercado,"avenida colonial 3046 la colonial, lima cercado",lima cercado,[Lima Cercado],Lima Cercado,Lima Cercado
3,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,NaN,...,lima cercado,"av. mariano cornejo 1848\nlima cercado, lima",True,Lima Cercado,Lima Cercado,"av. mariano cornejo 1848 lima cercado, lima",lima cercado,[Lima Cercado],Lima Cercado,Lima Cercado
4,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,lima cercado,"av. colonial 2140, lima 15081\nlima cercado, lima",True,Lima Cercado,Lima Cercado,"av. colonial 2140, lima 15081 lima cercado, lima",lima cercado,[Lima Cercado],Lima Cercado,Lima Cercado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51162,urbania,departamentos,venta,Lima Top,Barranco,"JR. Carlos Arrieta\nBarranco, Lima","S/ 521,640 · USD 144,900",521640.0,144900.0,NaN,...,barranco,"jr. carlos arrieta\nbarranco, lima",True,Barranco,Barranco,"jr. carlos arrieta barranco, lima",barranco,[Barranco],Barranco,Barranco
51163,urbania,departamentos,venta,Lima Top,Barranco,"Avenida Pedro De Osma 3\nBarranco, Lima","S/ 525,150 · USD 135,000",525150.0,135000.0,NaN,...,barranco,"avenida pedro de osma 3\nbarranco, lima",True,Barranco,Barranco,"avenida pedro de osma 3 barranco, lima",barranco,[Barranco],Barranco,Barranco
51164,urbania,departamentos,venta,Lima Top,Barranco,"AV GRAU 1100\nBarranco, Lima","S/ 609,000 · USD 175,000",609000.0,175000.0,NaN,...,barranco,"av grau 1100\nbarranco, lima",True,Barranco,Barranco,"av grau 1100 barranco, lima",barranco,[Barranco],Barranco,Barranco
51165,urbania,departamentos,venta,Lima Top,Barranco,"republica de panama 257\nBarranco, Lima","S/ 300,000 · USD 87,000",300000.0,87000.0,NaN,...,barranco,"republica de panama 257\nbarranco, lima",True,Barranco,Barranco,"republica de panama 257 barranco, lima",barranco,[Barranco],Barranco,Barranco


In [97]:
data_vivienda.columns

Index(['fuente', 'inmueble', 'operacion', 'zona', 'distrito', 'direccion',
       'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito_2',
       'distrito_norm', 'direccion_norm', 'distrito_en_direccion',
       'distrito_oficial', 'distrito2', 'dir_norm', 'dist_norm',
       'dist_candidates', 'distrito_candidato', 'distrito_3'],
      dtype='object')

In [100]:

print("\nCreacion de variables!")

cols_analisis = ['fuente','inmueble','operacion',
                'distrito_oficial', 'zona',
                'direccion','precio',
                'soles','dolares','variacion','mantenimiento','caracteristica',
                'area','dormitorio','baños','estacionamientos','detalle','enlace',
                'fecha','file_origin']
data_analisis = data_vivienda[cols_analisis].copy()
data_analisis


Creacion de variables!


,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,mantenimiento,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,120.0,66 m² tot.\n3 dorm.\n2 baños,66,3,2.0,NaN,Bonito departamento de estreno ubicado en Av. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,NaN,180.0,70 m² tot.\n3 dorm.\n2 baños,70,3,2.0,NaN,Alquiler de Departamento en Cercado• Av. Venez...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,NaN,160.0,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,NaN,300.0,70 m² tot.\n3 dorm.\n2 baños,70,3,2.0,NaN,Moderno departamento con buenos acabados en pi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,NaN,68 m² tot.\n3 dorm.\n2 baños\n1 estac.,68,3,2.0,1.0,Alquiler de departamento con cochera en Centra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51162,urbania,departamentos,venta,Barranco,Lima Top,"JR. Carlos Arrieta\nBarranco, Lima","S/ 521,640 · USD 144,900",521640.0,144900.0,NaN,NaN,55 m² tot.\n2 dorm.\n2 baños,55,2,2.0,NaN,Moderno edificio que contará con un total de 5...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv
51163,urbania,departamentos,venta,Barranco,Lima Top,"Avenida Pedro De Osma 3\nBarranco, Lima","S/ 525,150 · USD 135,000",525150.0,135000.0,NaN,280,45 m² tot.\n1 dorm.\n1 baño\n1 estac.,45,1,NaN,1.0,Vendo Lindo departamento en Avenida Pedro de O...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv
51164,urbania,departamentos,venta,Barranco,Lima Top,"AV GRAU 1100\nBarranco, Lima","S/ 609,000 · USD 175,000",609000.0,175000.0,NaN,300,75 m² tot.\n2 dorm.\n2 baños,75,2,2.0,NaN,"¡Vive en Monumento Grau, Barranco! ️ Exclusivo...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv
51165,urbania,departamentos,venta,Barranco,Lima Top,"republica de panama 257\nBarranco, Lima","S/ 300,000 · USD 87,000",300000.0,87000.0,NaN,170,42 m² tot.\n1 dorm.\n1 baño,42,1,NaN,NaN,Se vende departamento ideal para inversion de ...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv


In [102]:
# Normalizaciones básicas
data_analisis["fuente"] = data_analisis["fuente"].astype(str).str.strip().str.lower()
data_analisis["distrito_final"] = data_analisis["distrito_oficial"].astype(str).str.strip().str.title()
data_analisis["inmueble"] = data_analisis["inmueble"].astype(str).str.strip().str.lower()
data_analisis["operacion"] = data_analisis["operacion"].astype(str).str.strip().str.lower()
data_analisis["precio_pen"] = data_analisis["soles"]
data_analisis["precio_usd"] = data_analisis["dolares"]

## solo registros con precios
data_analisis["sin_precio"] = (data_analisis["precio_pen"].isna() & data_analisis["precio_usd"].isna()).astype(int)
data_analisis = data_analisis.query("sin_precio == 0").copy()

# AREA robusta
tmp = data_analisis["area"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mm = tmp.str.split(" a ", n=1, expand=True)
data_analisis["area_min"] = pd.to_numeric(mm[0].str.replace(",", "."), errors="coerce")
data_analisis["area_max"] = pd.to_numeric(mm[1].str.replace(",", "."), errors="coerce")
data_analisis["area_promedio"] = data_analisis[["area_min","area_max"]].mean(axis=1)

data_analisis

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,enlace,fecha,file_origin,distrito_final,precio_pen,precio_usd,sin_precio,area_min,area_max,area_promedio
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1800.0,NaN,0,66.0,NaN,66.0
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,2000.0,575.0,0,70.0,NaN,70.0
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1800.0,NaN,0,60.0,NaN,60.0
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,2000.0,NaN,0,70.0,NaN,70.0
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1800.0,NaN,0,68.0,NaN,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51162,urbania,departamentos,venta,Barranco,Lima Top,"JR. Carlos Arrieta\nBarranco, Lima","S/ 521,640 · USD 144,900",521640.0,144900.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv,Barranco,521640.0,144900.0,0,55.0,NaN,55.0
51163,urbania,departamentos,venta,Barranco,Lima Top,"Avenida Pedro De Osma 3\nBarranco, Lima","S/ 525,150 · USD 135,000",525150.0,135000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv,Barranco,525150.0,135000.0,0,45.0,NaN,45.0
51164,urbania,departamentos,venta,Barranco,Lima Top,"AV GRAU 1100\nBarranco, Lima","S/ 609,000 · USD 175,000",609000.0,175000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv,Barranco,609000.0,175000.0,0,75.0,NaN,75.0
51165,urbania,departamentos,venta,Barranco,Lima Top,"republica de panama 257\nBarranco, Lima","S/ 300,000 · USD 87,000",300000.0,87000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-03,data_venta_departamentos_urbania_Lima Top.csv,Barranco,300000.0,87000.0,0,42.0,NaN,42.0


In [103]:
# DORMITORIO robusto
tmpd = data_analisis["dormitorio"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mmd = tmpd.str.split(" a ", n=1, expand=True)
data_analisis["domitorio_min"] = pd.to_numeric(mmd[0], errors="coerce")
data_analisis["domitorio_max"] = pd.to_numeric(mmd[1], errors="coerce")

# Precio por m2 (división segura)
den = data_analisis["area_promedio"].replace(0, np.nan)
data_analisis["precio_m2_pen"] = (data_analisis["precio_pen"] / den).round(2)

# Flags
data_analisis["tiene_mantenimiento"] = data_analisis["mantenimiento"].notna().astype(int)
data_analisis["tiene_estacionamientos"] = data_analisis["estacionamientos"].notna().astype(int)
data_analisis["banios"] = data_analisis["caracteristica"].str.extract(r"(\d+)\s*bañ").astype("Int64")
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,sin_precio,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,0,66.0,NaN,66.0,3.0,NaN,27.27,1,0,2
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,NaN,...,0,70.0,NaN,70.0,3.0,NaN,28.57,1,0,2
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,NaN,...,0,60.0,NaN,60.0,3.0,NaN,30.00,1,0,2
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,NaN,...,0,70.0,NaN,70.0,3.0,NaN,28.57,1,0,2
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,0,68.0,NaN,68.0,3.0,NaN,26.47,0,1,2


In [104]:
data_analisis['distrito_categoria'] = data_analisis['zona'].astype('category')

In [105]:
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios,distrito_categoria
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,66.0,NaN,66.0,3.0,NaN,27.27,1,0,2,Lima Centro
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,NaN,...,70.0,NaN,70.0,3.0,NaN,28.57,1,0,2,Lima Centro
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,NaN,...,60.0,NaN,60.0,3.0,NaN,30.00,1,0,2,Lima Centro
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,NaN,...,70.0,NaN,70.0,3.0,NaN,28.57,1,0,2,Lima Centro
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,NaN,...,68.0,NaN,68.0,3.0,NaN,26.47,0,1,2,Lima Centro


In [ ]:
# Se crean columnas categóricas para los rangos de precio y área, que se usarán en los filtros de las pestañas.
data_analisis['precio_pen'] = pd.to_numeric(data_analisis['precio_pen'], errors='coerce')
data_analisis['precio_usd'] = pd.to_numeric(data_analisis['precio_usd'], errors='coerce')
data_analisis['area'] = pd.to_numeric(data_analisis['area'], errors='coerce')

In [114]:
data_analisis[data_analisis["operacion"]=="alquiler"]["precio_pen"].describe()

count    7.658000e+03
mean     8.388728e+03
std      8.478494e+04
min      4.000000e+00
25%      2.100000e+03
50%      2.900000e+03
75%      4.500000e+03
max      2.699370e+06
Name: precio_pen, dtype: float64

In [115]:
data_analisis[data_analisis["operacion"]=="venta"]["precio_pen"].describe()

count    4.337300e+04
mean     1.518108e+07
std      1.937261e+09
min      1.000000e+00
25%      4.950000e+05
50%      8.445600e+05
75%      1.596000e+06
max      2.852552e+11
Name: precio_pen, dtype: float64

In [116]:
# Bins y labels para precio de venta (Soles)
bins_alquiler = [-1, 1500, 2500, 3500, 4500, float('inf')]
labels_alquiler = ["Hasta S/1.5k", "De S/1.5k a S/2.5k", "De S/2.5 a S/3.5k", "De S/3.5k a S/4.5k", "De S/4.5k a más"]
data_analisis['precio_alquiler_agp'] = pd.cut(data_analisis[data_analisis["operacion"]=="alquiler"]["precio_pen"], bins=bins_alquiler, labels=labels_alquiler, right=False)

# Bins y labels para precio de venta (Dólares)
bins_venta = [-1, 50000, 100000, 200000, 500000, float('inf')]
labels_venta = ["Hasta $ 50k", "De $ 50k a $ 100k", "De $ 100k a $ 200k", "De $ 200k a $ 500k", "De $ 500k a más"]
data_analisis['precio_venta_agp'] = pd.cut(data_analisis[data_analisis["operacion"]=="venta"]['precio_usd'], bins=bins_venta, labels=labels_venta, right=False)

# Bins y labels para área (m²)
bins_area = [-1, 50, 100, 200, 300, float('inf')]
labels_area = ["Hasta 50m2", "De 50m2 a 100m2", "De 100m2 a 200m2", "De 200m2 a 300m2", "De 300m2 a más"]
data_analisis['area_agp'] = pd.cut(data_analisis['area'], bins=bins_area, labels=labels_area, right=False)

## Estacionamiento
data_analisis["estacionamiento_gp"] = data_analisis["estacionamientos"].apply(lambda x: "Si" if x > 0 else "No")

In [93]:
data_analisis["distrito_oficial"].value_counts()

distrito_oficial
miraflores                 8038
santiago de surco          7866
san isidro                 4847
san borja                  2258
la molina                  2105
barranco                   1694
san miguel                 1281
surquillo                   863
chorrillos                  725
jesus maria                 695
magdalena                   631
pueblo libre                577
lima cercado                574
lince                       516
ate vitarte                 408
san martin de porres        365
los olivos                  345
la victoria                 282
comas                       279
punta hermosa               275
san juan de lurigancho      260
carabayllo                  218
san bartolo                 214
san juan de miraflores      184
brena                       131
lurin                       113
chosica lurigancho           96
rimac                        93
puente piedra                91
villa el salvador            90
santa maria del mar    

In [121]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito_oficial', 'zona',
       'direccion', 'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito_final',
       'precio_pen', 'precio_usd', 'sin_precio', 'area_min', 'area_max',
       'area_promedio', 'domitorio_min', 'domitorio_max', 'precio_m2_pen',
       'tiene_mantenimiento', 'tiene_estacionamientos', 'banios',
       'distrito_categoria', 'precio_alquiler_agp', 'precio_venta_agp',
       'area_agp', 'estacionamiento_gp'],
      dtype='object')

In [123]:
# Orden final
cols_final = [
    'fuente','inmueble','operacion', 'distrito_oficial', 'zona','direccion',
    'precio','soles','dolares','precio_pen','precio_usd','variacion',
    'mantenimiento','tiene_mantenimiento','caracteristica',
    'area','area_min','area_max','area_promedio', 'area_agp', 'precio_m2_pen', 'precio_venta_agp', 'precio_alquiler_agp',
    'dormitorio','domitorio_min','domitorio_max','banios', 'estacionamientos','tiene_estacionamientos', 'estacionamiento_gp',
    'detalle','enlace','fecha','file_origin'
]
data_analisis = data_analisis[cols_final].copy()

In [125]:
data_analisis.head(100)

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,precio_pen,...,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,1800.0,...,3.0,NaN,2,NaN,0,No,Bonito departamento de estreno ubicado en Av. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,2000.0,...,3.0,NaN,2,NaN,0,No,Alquiler de Departamento en Cercado• Av. Venez...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,1800.0,...,3.0,NaN,2,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,2000.0,...,3.0,NaN,2,NaN,0,No,Moderno departamento con buenos acabados en pi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,1800.0,...,3.0,NaN,2,1.0,1,Si,Alquiler de departamento con cochera en Centra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"CALLE FERMIN TANGUIS , LA VICTORIA\nTupac Amar...","S/ 2,200 · USD 654",2200.0,654.0,2200.0,...,2.0,NaN,2,NaN,0,No,Pg/ Te presento un moderno departamento en est...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
98,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"JAVIER PRADO ESTE 1091\nSanta Catalina, La Vic...","S/ 3,300 · USD 975",3300.0,975.0,3300.0,...,3.0,NaN,2,1.0,1,Si,Alquiler de Departamento de estreno( se alquil...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
99,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"AV SOLIDARIDAD 103\nSanta Catalina, La Victoria","S/ 2,300 · USD 670",2300.0,670.0,2300.0,...,2.0,NaN,2,1.0,1,Si,¡amplio Y lindo dpto de 2 habitaciones en el p...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
100,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"PASEO DE LA REPUBLICA\nLa Victoria, Lima","S/ 2,500",2500.0,NaN,2500.0,...,3.0,NaN,2,NaN,0,No,Alquilo moderno departamento C/vista externa e...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...


In [126]:
print("\nData Final creada satisfacotiramente!")

# Export
out_path = DATA_OUT / "data_dondevivir_analisis.csv"
data_analisis.to_csv(out_path, sep="|", index=False)
print("\nData Final creada satisfactoriamente ->", out_path)


Data Final creada satisfacotiramente!

Data Final creada satisfactoriamente -> C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed\data_dondevivir_analisis.csv


In [127]:
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,precio_pen,...,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,1800.0,...,3.0,NaN,2,NaN,0,No,Bonito departamento de estreno ubicado en Av. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,575.0,2000.0,...,3.0,NaN,2,NaN,0,No,Alquiler de Departamento en Cercado• Av. Venez...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,NaN,1800.0,...,3.0,NaN,2,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,NaN,2000.0,...,3.0,NaN,2,NaN,0,No,Moderno departamento con buenos acabados en pi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,NaN,1800.0,...,3.0,NaN,2,1.0,1,Si,Alquiler de departamento con cochera en Centra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
